# First image manipulation ideas (not in web-app)

### `create_rotate_image`

In [13]:
def create_rotate_image(angle: int):
    def rotate_image(image: np.ndarray):
        pilImage = Image.fromarray(image)
        pilImage = pilImage.rotate(angle=angle)
        return np.array(pilImage)
    return rotate_image

### `flip_image_horizontally`
The function takes a NumPy array representing an image as input, converts it to a PIL Image, flips the image horizontally using the transpose() method with the specified transformation (FLIP_LEFT_RIGHT), and then converts the horizontally flipped PIL Image back to a NumPy array before returning the result.

In [14]:
def flip_image_horizontally(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.transpose(Image.FLIP_LEFT_RIGHT)
    return np.array(pilImage)

NameError: name 'np' is not defined

### `change_to_grayscale`
The function accepts a NumPy array representing an image as input, transforms it into a PIL Image, converts the color image to grayscale using the convert() method with the 'L' mode, and then converts the resulting grayscale PIL Image back to a NumPy array before returning the processed image.

In [ ]:
def change_to_grayscale(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.convert('L')
    return np.array(pilImage)

### `apply_gauss_noise`
The function generates Gaussian noise with a mean of 50 and a standard deviation of 10 using NumPy's random.normal() function. This noise is created to match the shape of the input image. The generated noise is then added to the original image using the OpenCV add() function, resulting in an image with applied Gaussian noise.

In [ ]:
def apply_gauss_noise(image: np.ndarray):
    gauss = np.random.normal(50, 10, image.shape).astype('uint8')
    return cv2.add(image, gauss)

### `blur_edges`

In [ ]:
def blur_edges(image: np.ndarray):
    # edge detection
    grey_scale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    horizontalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]), mode='same', boundary='symm')
    verticalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]), mode='same', boundary='symm')
    plt.matshow(horizontalEdgeImage, cmap='gray')
    plt.show()
    plt.matshow(verticalEdgeImage, cmap='gray')
    plt.show()
    edge_image = pow(pow(horizontalEdgeImage, 2) + pow(verticalEdgeImage, 2), 0.5)
    plt.matshow(edge_image, cmap='gray')
    plt.show()

    plt.matshow(np.abs(edge_image), cmap='gray')
    plt.show()

    print(edge_image.shape)
    edge_image_all_colors = np.tile(edge_image, (1, 3))

    print(edge_image_all_colors.shape)
    edge_image_all_colors_reshaped = edge_image_all_colors.reshape(edge_image.shape[0], edge_image.shape[1], 3)

    print(edge_image_all_colors_reshaped.shape)
    image_edge_blur = image + ((edge_image_all_colors_reshaped - image) * 0.2).astype('uint8')
    plt.matshow(image_edge_blur)
    plt.show()

    laplacian = cv2.Laplacian(grey_scale_image, cv2.CV_64F)
    plt.matshow(laplacian, cmap='gray')
    plt.show()

    plt.matshow(np.abs(laplacian), cmap='gray')
    plt.show()

    return image_edge_blur

### `create_add_black_squares`
The outer function, `create_add_black_squares`, takes two integer parameters, `square_size` and `number_of_squares`, and returns an inner function `add_black_squares`. When this inner function is called with a NumPy array representing an image, it iteratively adds black squares to the image. The size and number of squares are determined by the parameters provided during the creation of the outer function. The positions of the black squares are randomly generated within the image dimensions, and the pixel values within the specified square regions are set to 0 (black). The resulting image is then adjusted to ensure pixel values remain within the valid 0-255 range before being returned.

In [ ]:
def create_add_black_squares(square_size: int, number_of_squares: int):
    def add_black_squares(image: np.ndarray):
        for _ in range(number_of_squares):
            x = np.random.randint(0, image.shape[0] - square_size)
            y = np.random.randint(0, image.shape[1] - square_size)
            image[x:x + square_size, y:y + square_size] = 0  # 0 because squares are black

        # Ensure the image values stay within 0-255 range
        return np.clip(image, 0, 255).astype(np.uint8)

    return add_black_squares

### `apply_bilateral_filter`
The function applies a bilateral filter to the input image using the OpenCV `bilateralFilter` function. The bilateral filter is a non-linear, edge-preserving smoothing filter that considers both spatial and intensity differences between pixels. The parameters used for this filter are set to 9 for the diameter of the pixel neighborhood, and 75 for both the color and spatial sigma values. These parameters control the extent of filtering in terms of pixel proximity and color similarity. The resulting filtered image is then returned as the output of the function.

In [ ]:
def apply_bilateral_filter(image):
    # bilateral filter (explain params)
    filtered_image = cv2.bilateralFilter(image, 9, 75, 75)
    return filtered_image

# Additional Implemented image manipulation ideas

### `apply_dithering`
The dithering process in this filter reduces the color range of the image to a predefined palette, in this case, 16 colors. It achieves this through a quantization process that groups similar colors, followed by a conversion back to an RGB format. The outcome is an image characterized by distinct color blocks and a notable reduction in color gradation. This effect can be applied globally to the image or localized to a region determined by facial keypoints.

In [ ]:
def apply_dithering(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.quantize(colors=16).convert('RGB')
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_max_filter`
This filter operates by scanning the image and replacing each pixel with the maximum pixel value in its neighborhood, defined by a specified filter size. As a result, brighter areas within the filter's radius become more prominent, while darker regions are subdued. This enhances the luminance contrast and can accentuate certain features of the image. The filter can be applied to the entire image or restricted to a region identified by facial keypoints.

In [ ]:
def apply_max_filter(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.filter(ImageFilter.MaxFilter(9))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_min_filter`
This filter is the inverse of the Max Filter. It scans the image and replaces each pixel with the minimum pixel value in its neighborhood. This process emphasizes darker areas and reduces the prominence of brighter ones. The Min Filter accentuates shadows and darker regions, providing a contrasting effect to the Max Filter. It can be used across the whole image or targeted to a specific area using facial keypoints.

In [15]:
def apply_min_filter(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.filter(ImageFilter.MinFilter(9))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

NameError: name 'Image' is not defined

### `apply_closing`
The Closing Filter combines the effects of the Min and Max Filters in sequence. Initially, it applies the Min Filter, which reduces the prominence of smaller bright areas, followed by the Max Filter, which enhances the surrounding brighter regions. This sequence effectively 'closes' small gaps and dark spots, resulting in a smoother appearance in bright areas of the image. This filter can be applied universally or selectively based on keypoints.

In [ ]:
def apply_closing(image: Image, keypoints, only_face=True) -> Image:
    modified_image = apply_min_filter(apply_max_filter(image, keypoints, False), keypoints, False)
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_opening`
The Opening Filter reverses the sequence of the Closing Filter. It starts with the Max Filter, which diminishes small dark regions, followed by the Min Filter, which then reduces the surrounding darker areas. This 'opening' effect is particularly noticeable in darker parts of the image, creating a sense of expansion in these regions. The filter can be applied to the entire image or localized to a specific area using keypoints.

In [ ]:
def apply_opening(image: Image, keypoints, only_face=True) -> Image:
    modified_image = apply_max_filter(apply_min_filter(image, keypoints, False), keypoints, False)
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_color_shift`
This filter introduces a random shift in the color channels of the image. It independently alters the red, green, and blue channels by a random value within a specified range. The effect of this is a shift in the overall color balance of the image, producing a variety of color tones and hues. The degree of color shift is controlled by the max_shift_intensity parameter. The filter can modify the entire image or be confined to a particular region, such as the face, based on keypoints.

In [ ]:
def apply_color_shift(image: Image, keypoints, only_face=True, max_shift_intensity=25) -> Image:
    r_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    g_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    b_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    r, g, b = image.split()
    r = r.point(lambda i: i + r_shift)
    g = g.point(lambda i: i + g_shift)
    b = b.point(lambda i: i + b_shift)
    modified_image = Image.merge('RGB', (r, g, b))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_whole_face_mask`
This filter applies a face mask image over the entire face. It uses facial keypoints to determine the position and orientation of the face. The mask is resized to match the width of the face and rotated to align with the angle between the eyes. The mask is then placed over the face, covering it entirely, simulating the effect of wearing a full-face mask.

In [ ]:
def apply_whole_face_mask(image: Image, keypoints) -> Image:
    foreground = Image.open('filters/whole_face_mask.png').convert("RGBA")
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        left_eye = face_keypoints['left_eye']
        right_eye = face_keypoints['right_eye']
        dx = right_eye[0] - left_eye[0]
        dy = right_eye[1] - left_eye[1]
        angle_radians = math.atan2(-dy, dx)
        angle_degrees = math.degrees(angle_radians)
        face_width = box[2]
        foreground_width_to_height_ratio = foreground.size[0] / foreground.size[1]
        foreground = foreground.resize(size=(face_width, int(face_width / foreground_width_to_height_ratio)))
        rotated_overlay = foreground.rotate(angle_degrees, expand=True)
        left_upper_face_mask = (box[0], box[1])
        left_upper_paste = (left_upper_face_mask[0], int(left_upper_face_mask[1] - math.fabs(
            math.cos(math.radians(90 - angle_degrees)) * face_width)))
        image.paste(rotated_overlay, left_upper_paste, rotated_overlay)
    return image

### `apply_highlight_keypoints`
This filter visually highlights the facial keypoints and connections between them on the image. It draws circles around each keypoint and lines connecting them, using different colors for different facial features. This filter serves to visually emphasize the positions and relationships of facial features as identified by the keypoints.

In [ ]:
def apply_highlight_keypoints(image: Image, keypoints) -> Image:
    draw = ImageDraw.Draw(image)
    if len(keypoints) > 0:
        for keypoint_set in keypoints:
            for j in range(len(keypoint_set[2])):
                x, y = keypoint_set[2][j]
                if j < len(keypoint_set[2]) - 1:
                    next_x, next_y = keypoint_set[2][j + 1]
                    draw.line((x, y, next_x, next_y), fill='lightgreen', width=3)
                radius = 5
                draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill='green', outline='lightgreen')
            for feature, coords in keypoint_set[1].items():
                x, y = coords
                radius = 10
                draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill='red', outline='red')
    return image

### `apply_distance_transformation`
This filter applies a distance transformation technique to an image. It starts by converting the image to grayscale and then to a binary format based on a specified threshold (in this case, 128). Binary dilation and erosion operations are performed on the binary image to highlight regions of change. Dilation expands the white areas, while erosion shrinks them. The filter then compares the dilated and eroded images, highlighting the differences with white pixels. The result is an image that emphasizes the structural changes in the original image, providing a unique visual representation of distance transformation.

In [16]:
def apply_distance_transformation(image: Image) -> Image:
    image_np = np.array(image.convert('L'))
    threshold = 128
    binary_image = (image_np > threshold).astype(np.uint8)
    dilated = binary_dilation(binary_image, iterations=5)
    eroded = binary_erosion(binary_image, iterations=5)
    morphed_image = np.where(dilated != eroded, 255, 0).astype(np.uint8)
    return Image.fromarray(morphed_image).convert('RGB')

NameError: name 'Image' is not defined

### `apply_vertical_edge`
 This filter emphasizes vertical edges in an image using a specific kernel in a convolution process. The kernel used is designed to respond strongly to vertical lines or edges by subtracting the pixel value on the left from the pixel value on the right. This operation enhances vertical features while diminishing horizontal features. The filter is particularly effective in highlighting vertical structures or details in an image, making them more pronounced against the background.

In [ ]:
def apply_vertical_edge(image: Image) -> Image:
    return image.filter(ImageFilter.Kernel((3, 3), (-1, 0, 1, -2, 0, 2, -1, 0, 1), 1, 0))

### `apply_horizontal_edge`
Similar to the Vertical Edge Filter, this filter is designed to accentuate horizontal edges in an image. It uses a convolution kernel that contrasts pixel values above and below each pixel in the image. By doing so, horizontal lines or edges become more pronounced. This filter is useful for emphasizing horizontal features or details within the image, enhancing their visibility and distinction from the rest of the image elements.

In [ ]:
def apply_horizontal_edge(image: Image) -> Image:
    return image.filter(ImageFilter.Kernel((3, 3), (-1, -2, -1, 0, 0, 0, 1, 2, 1), 1, 0))